In [5]:
from openpyxl import load_workbook  # 导入load_workbook函数
import pandas as pd  # 导入pandas库
import os  # 导入os库

# 加载工作簿
workbook_path = r'Problem D Great Lakes.xlsx'  # 指定工作簿路径
workbook = load_workbook(workbook_path)  # 使用load_workbook函数加载工作簿

# 用于存储生成的CSV文件路径的列表
generated_csv_paths = []

# 处理每个工作表
for sheet_name in workbook.sheetnames:  # 遍历工作簿中的每个工作表名称
    # 加载工作表
    sheet = workbook[sheet_name]  # 使用工作表名称获取工作表对象
    
    # 提取第一行作为文件名
    filename_row = sheet[1]  # 获取第一行数据
    filename = '_'.join([cell.value for cell in filename_row if cell.value is not None]) + '.csv'  # 构建文件名
    csv_path = os.path.join('./', filename)  # 构建CSV文件的路径
    
    # 将工作表转换为DataFrame，跳过第一行（表头）并保留行数从7到30
    df = pd.DataFrame(sheet.values)  # 将工作表数据转换为DataFrame
    df = df.iloc[6:30]  # 行是从0开始索引的，因此第7行在索引6处
    
    # 使用工作表的第二行作为列名
    df.columns = df.iloc[0]  # 将第二行数据设置为DataFrame的列名
    df = df[1:]  # 删除用作表头的行
    
    # 保存为CSV文件
    df.to_csv(csv_path, index=False)  # 将DataFrame保存为CSV文件，不包括索引列
    generated_csv_paths.append(csv_path)  # 将生成的CSV文件路径添加到列表中

generated_csv_paths  # 显示生成的CSV文件路径列表


['./Lake Superior - Mean Water Level.csv',
 "./St. Mary's River - Flow .csv",
 './Lake Michigan and Lake Huron - Mean Water Level.csv',
 './St. Clair River - Flow .csv',
 './Lake St. Clair - Mean Water Level.csv',
 './Detroit River - Flow .csv',
 './Lake Erie - Mean Water Level.csv',
 './Niagara River - Flow at Buffalo.csv',
 './Lake Ontario - Mean Water Level.csv',
 './Ottawa River - Flow at Carillon.csv',
 './St. Lawrence River - Flow at Cornwall.csv']

In [6]:
# 初始化一个空的DataFrame，用于合并的数据集
merged_df = pd.DataFrame()

# 循环遍历每个CSV文件的路径，将其追加到合并的DataFrame中
for csv_path in generated_csv_paths:  # 遍历生成的CSV文件路径列表
    # 读取CSV文件
    df = pd.read_csv(csv_path)  # 使用pd.read_csv函数读取CSV文件
    
    # 从文件名中提取标识符（去除“.csv”之前的部分）
    identifier = os.path.basename(csv_path).replace('.csv', '')  # 从文件路径中提取文件名，并去除“.csv”
    
    # 将标识符作为新列添加到DataFrame中
    df['Source'] = identifier  # 将标识符添加为名为“Source”的新列
    
    # 追加到合并的DataFrame中
    merged_df = pd.concat([merged_df, df], ignore_index=True)  # 使用pd.concat函数将DataFrame追加到合并的DataFrame中，忽略索引

# 显示合并后的DataFrame的一部分以进行验证
merged_df.head()  # 显示合并后的DataFrame的前几行数据以验证合并结果


,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Source,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,2000,183.16,183.08,183.08,183.12,183.16,183.24,183.33,183.31,183.27,183.2,183.15,183.06,Lake Superior - Mean Water Level,NaN,NaN,NaN
1,2001,182.98,182.94,182.91,183.01,183.24,183.33,183.36,183.39,183.37,183.3,183.29,183.3,Lake Superior - Mean Water Level,NaN,NaN,NaN
2,2002,183.22,183.14,183.11,183.14,183.26,183.31,183.36,183.4,183.42,183.44,183.36,183.26,Lake Superior - Mean Water Level,NaN,NaN,NaN
3,2003,183.16,183.07,183.03,183.08,183.18,183.23,183.26,183.29,183.27,183.26,183.23,183.19,Lake Superior - Mean Water Level,NaN,NaN,NaN
4,2004,183.14,183.07,183.07,183.13,183.23,183.35,183.4,183.42,183.46,183.47,183.46,183.38,Lake Superior - Mean Water Level,NaN,NaN,NaN


In [7]:
merged_df[['Source','Year', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep',
       'Oct', 'Nov', 'Dec']].to_csv('data.csv',index=None)

In [8]:
# 重新加载已经清理并存储的合并数据集
merged_df_cleaned = pd.read_csv('data.csv', na_values='---')  # 使用pd.read_csv函数重新加载数据集，将'---'替换为NaN

# 对于每个 'Source'，将 '---' 替换为 NaN，然后计算每列的均值，排除非数字列如 'Year' 和 'Source'
numeric_columns = merged_df_cleaned.select_dtypes(include=['float64', 'int64']).columns.difference(['Year'])  # 选择数值列，排除 'Year' 列

# 将 '---' 替换为 NaN 并根据 'Source' 填充均值
for source in merged_df_cleaned['Source'].unique():  # 遍历每个 'Source'
    # 计算当前 'Source' 的每个数值列的均值，排除NaN值
    means = merged_df_cleaned[merged_df_cleaned['Source'] == source][numeric_columns].mean()  # 计算每列的均值
    # 使用当前 'Source' 的计算均值来填充NaN值
    for column in numeric_columns:  # 遍历数值列
        merged_df_cleaned.loc[merged_df_cleaned['Source'] == source, column] = merged_df_cleaned[merged_df_cleaned['Source'] == source][column].fillna(means[column])

# 通过检查是否还有 '---' 来验证替换是否成功
merged_df_cleaned.isin(['---']).sum().sum()  # 检查是否还有 '---'，如果返回0则表示替换成功


0

In [9]:
merged_df_cleaned.to_csv('data.csv',index=None)